In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt

TREND_NUM = 50

In [ ]:
#喜歡數與不喜歡數
def tumb_up_n_down(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    
    tumb_up = soup.select('.yt-uix-button-content')[15]
    tumb_up = tumb_up.text.replace(',','')
    
    tumb_down = soup.select('.yt-uix-button-content')[16]
    tumb_down = tumb_down.text.replace(',','')
    
    if (('顯示' in tumb_up) or ('顯示' in tumb_down)):
        tumb = [-1,-1]
    else:
        tumb = [int(tumb_up),int(tumb_down)]
    return tumb

In [ ]:
def Video_type(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    
    ty = -1
    term = 4
    
    while(ty is -1):
        video_type = soup.select('.spf-link')[term].text
        
        if video_type in "汽車與交通工具" :
            ty = 0
        elif video_type in "時裝及美容" :
            ty = 1
        elif video_type in "喜劇" :
            ty = 2
        elif video_type in "教育" :
            ty = 3
        elif video_type in "娛樂" :
            ty = 4
        elif video_type in "家庭娛樂" :
            ty = 5
        elif video_type in "電影與動畫":
            ty = 6
        elif video_type in "美食":
            ty = 7
        elif video_type in "遊戲":
            ty = 8
        elif video_type in "DIY 教學":
            ty = 9
        elif video_type in "音樂":
            ty = 10
        elif video_type in "新聞與政治":
            ty = 11
        elif video_type in "非營利組織與社運活動":
            ty = 12
        elif video_type in "人物與網誌":
            ty = 13
        elif video_type in "寵物與動物":
            ty = 14
        elif video_type in "科學與科技":
            ty = 15
        elif video_type in "體育":
            ty = 16
        elif video_type in "旅行與活動":
            ty = 17
        else:
            term += 1

    return ty

In [ ]:
url = "https://www.youtube.com/feed/trending"
res = requests.get(url)
soup = BeautifulSoup(res.text)

titles = [] #影片標題
views = [] #觀看次數
times = [] #上傳時間
hrefs = [] #影片連結
video_times = [] #片長
video_times_sec = [] #片長sec

In [ ]:
print(soup.select('.yt-lockup-content')[27].select('.spf-link')[0].text)

In [ ]:
for value in soup.select('.yt-lockup-content'):
    title = value.select('.spf-link')[0]
    titles.append(title.text)
    
    href = value.find('a')['href']
    href = 'https://www.youtube.com' + href
    hrefs.append(href)
    
    view = value.select('li')[1]
    view = view.text
    view = view.split('：')[1]
    view = view.split('次')[0]
    view = view.replace(',','')
    views.append(int(view))
    
    time = value.select('li')[0].text
    times.append(time)
    
    vedio_time = value.select('span')[0].text
    vedio_time = vedio_time.split('播放時間：')[1]
    vedio_time = vedio_time.split('。')[0]
    vedio_time = vedio_time.split(':')
    video_times.append(vedio_time)

In [ ]:
for item in video_times:
    if len(item) is 3:
        #print(int(item[0])*3600+int(item[1])*60+int(item[2]))
        video_times_sec.append(int(item[0])*3600+int(item[1])*60+int(item[2]))
    elif len(item) is 2:
        #print(int(item[0])*60+int(item[1]))
        video_times_sec.append(int(item[0])*60+int(item[1]))

In [ ]:
#titles
#views
#times
#hrefs
#print(video_times_sec)

In [ ]:
#標記上傳時間 分類
time_mark = []
for item in times:
    if ('剛剛' or '分鐘' or '秒鐘') in item:
        time_mark.append('0')
    elif '小時' in item:
        time_mark.append('1')
    elif '天' in item:
        time_mark.append('2')
    elif '週' in item:
        time_mark.append('3')
    elif '月' in item:
        time_mark.append('4')
    elif '年' in item:
        time_mark.append('5')

In [ ]:
#time_mark
#test = []
#test.append(Video_type('https://www.youtube.com/watch?v=23KqaSIib9k'))
#print(test)
#print(type(test[0]))

In [ ]:
trend_view_info = np.zeros(shape=(TREND_NUM,6))

values = range(TREND_NUM)
with tqdm(total=len(values)) as pbar:
    for idx,item in enumerate(hrefs):
        if idx < TREND_NUM:
            trend_view_info[idx] = [tumb_up_n_down(item)[0],tumb_up_n_down(item)[1],views[idx],\
                                    tumb_up_n_down(item)[0]/views[idx],Video_type(item),video_times_sec[idx]]
            #print(tumb_up_n_down(item))
            pbar.write('%d/%d' %(idx+1,TREND_NUM))
            pbar.update(1)      
pbar.close()

In [ ]:
#np.log10(trend_view_info[:,5])/4

In [ ]:
# Compute pie slices
theta = trend_view_info[:,4]*(np.pi/9)
radii = np.arange( 50, 0, -1)
width = (np.pi/1.5) * trend_view_info[:,3]

plt.figure(figsize=(10, 10))
ax = plt.subplot(111, projection='polar')
ax.set_xticklabels(['Auto & Vehicles','Beauty & Fashion','Comedy','Education','Entertainment','Family Entertainment',\
                    'Film & Animation','Food','Gaming','How-to & Style','Music','News & Politics','Nonprofits & Activism'\
                    ,'People & Blogs','Pets & Animals','Science & Technology','Sports','Travel & Events'])
ax.set_xticks((np.pi/9)*np.arange( 0, 18, 1))
ax.set_rticks([])
bars = ax.bar(theta, radii, width=width, bottom=0)

# Use custom colors and opacity
for r, bar in zip(trend_view_info[:,5], bars):
    #print(r)
    bar.set_facecolor(plt.cm.gnuplot2(np.log10(r)/4))
    bar.set_alpha(0.6)

ax.set_title("YouTube trending categories in Taiwan", fontsize=20, fontweight='bold')
plt.show()